In [67]:
require 'cudnn'
require 'inn'
require 'image'
require 'dcnn'
Plot = require 'itorch.Plot'

-- Loads the mapping from net outputs to human readable labels
function load_synset()
  local file = io.open 'synset_words.txt'
  local list = {}
  while true do
    local line = file:read()
    if not line then break end
    table.insert(list, string.sub(line,11))
  end
  return list
end

-- Converts an image from RGB to BGR format and subtracts mean
function preprocess(im, img_mean)
  -- rescale the image
  local im3 = image.scale(im,224,224,'bilinear')*255
  -- RGB2BGR
  local im4 = im3:clone()
  im4[{1,{},{}}] = im3[{3,{},{}}]
  im4[{3,{},{}}] = im3[{1,{},{}}]

  -- subtract imagenet mean
  return im4 - image.scale(img_mean, 224, 224, 'bilinear')
end

-- Lua implementation of PHP scandir function
function scandir(directory)
    local i, t, popen = 0, {}, io.popen
    for filename in popen('ls -a "'..directory..'"'):lines() do
        if filename:match "%.png$" then
            i = i + 1
            t[i] = filename

        end
    end
    return t, i
end


fileDir = '/usr/local/data/imagenet/train'
fileList, total_files = scandir( fileDir )




-- Setting up networks 
print '==> Loading network'

--net = torch.load('/home/yusheng/Workspace/DeepLearning/models/zeilerNet/zeilerNet.net')
net = torch.load('/usr/local/data/zeilerNet/zeilerNet.net')
--net:add(nn.Sigmoid())
net:cuda()
-- as we want to classify, let's disable dropouts by enabling evaluation mode
net:evaluate()


print '==> Loading synsets'
synset_words = load_synset()

print '==> Loading image and imagenet mean'
image_name = 'Goldfish3.jpg'
--image_name = 'lena.jpg'
--image_name='people2.jpg'
img_mean_name = 'ilsvrc_2012_mean.t7'

im = image.load(image_name)
img_mean = torch.load(img_mean_name).img_mean:transpose(3,1)




-- Have to resize and convert from RGB to BGR and subtract mean
print '==> Preprocessing'
I = preprocess(im, img_mean)

sorted_prob, classes = net:forward(I:cuda()):view(-1):float():sort(true)

for i=1,5 do
  print('predicted class '..tostring(i)..': ', synset_words[classes[i]] .. ' ' .. sorted_prob[i] )
end

true_label = classes[1]

print(sorted_prob[1])

==> Loading network	


==> Loading synsets	
==> Loading image and imagenet mean	


==> Preprocessing	


predicted class 1: 	goldfish, Carassius auratus 21.853404998779	
predicted class 2: 	macaw 11.892498016357	
predicted class 3: 	flamingo 11.434491157532	
predicted class 4: 	axolotl, mud puppy, Ambystoma mexicanum 11.326101303101	
predicted class 5: 	eft 10.631513595581	
21.853404998779	


In [68]:
-- function NormalizedMax( fc_sorted_output ) 
--     local output_size = fc_sorted_output:size(1)
--     local normalized_output = {}
--     local sum = 0
    
--     for i=1, output_size  do
--         normalized_output[i] = fc_sorted_output[i]-fc_sorted_output[output_size]
--         sum = sum + normalized_output[i]
--     end
    
--     return normalized_output[1], sum/output_size
-- end

-- local average = 0
-- output_org, average = NormalizedMax( sorted_prob )
-- local ratio_of_mean_error = (output_org - average)/average 


-- print(output_org)
-- print(average)
-- print(ratio_of_mean_error)

In [69]:
-- local nThreads = 4
classifier = nn.Sequential()
classifier:add(net:get(2))
classifier:add(net:get(3))
--classifier:add(net:get(4))

local n=256

classification_curve = torch.Tensor(256)

--for n=1, 256 do
    topN_index = {}
    for i=1, n do
        topN_index[i] = { idx=0, diff=0 }
    end

    for i=1, net:get(1):get(13).output:size(1) do
        local incomplete_fm = net:get(1):get(13).output:clone()
        incomplete_fm[i]:zero()
        sorted_prob, classes = classifier:forward(incomplete_fm):view(-1):float():sort(true)
        --local diff = output_org - NormalizedMax( sorted_prob )
        local diff = output_org - sorted_prob[1]


        if topN_index[n].diff == 0 then 
            topN_index[n].diff = diff
            topN_index[n].idx = i

            local idx = n-1
            while idx>=1 and topN_index[idx].diff == 0 do
                topN_index[idx].diff = topN_index[idx+1].diff
                topN_index[idx].idx = topN_index[idx+1].idx
                topN_index[idx+1].diff = 0
                idx = idx - 1
            end
        
            if n==256 then
                for j=1,n-1 do 
                    for k=n,j+1,-1 do
                        -- print(k ..' '.. j )
                        if topN_index[k].diff > topN_index[k-1].diff then  
                            temp_diff = topN_index[k].diff
                            temp_idx = topN_index[k].idx
                            topN_index[k].diff = topN_index[k-1].diff
                            topN_index[k].idx = topN_index[k-1].idx
                            topN_index[k-1].diff = temp_diff
                            topN_index[k-1].idx = temp_idx
                        end
                    end
                end
            end
        else
            local temp_diff = 0
            local temp_idx = 0
            if n>1 then
                for j=1,n-1 do 
                    for k=n,j+1,-1 do
                        -- print(k ..' '.. j )
                        if topN_index[k].diff > topN_index[k-1].diff then  
                            temp_diff = topN_index[k].diff
                            temp_idx = topN_index[k].idx
                            topN_index[k].diff = topN_index[k-1].diff
                            topN_index[k].idx = topN_index[k-1].idx
                            topN_index[k-1].diff = temp_diff
                            topN_index[k-1].idx = temp_idx
                        end
                    end
                end
            end


    --         for j=1,5 do
    --             print( i .. '   ' .. top5_index[j].diff )
    --         end

            if topN_index[n].diff < diff then
                topN_index[n].diff = diff
                topN_index[n].idx = i
            end
        end

        --for i=1,5 do
        --print('predicted class '..tostring(i)..': ', synset_words[classes[1]] .. ' ' .. sorted_prob[1] )
        --end
    end


    for i=1,n do
        print( 'Top'.. i ..' drop down : ' .. topN_index[i].idx .. ' Diff: ' .. topN_index[i].diff) 
    end
    --print(incomplete_fm[1])
    --itorch.image(incomplete_fm)
    


Top1 drop down : 153 Diff: 12.442556858063	
Top2 drop down : 12 Diff: 12.233427524567	
Top3 drop down : 179 Diff: 11.673161029816	
Top4 drop down : 246 Diff: 9.2673306465149	
Top5 drop down : 154 Diff: 9.167902469635	
Top6 drop down : 57 Diff: 8.9293885231018	
Top7 drop down : 74 Diff: 8.8669819831848	
Top8 drop down : 123 Diff: 8.8308167457581	
Top9 drop down : 163 Diff: 8.7803235054016	
Top10 drop down : 167 Diff: 8.5559697151184	
Top11 drop down : 196 Diff: 8.4277310371399	
Top12 drop down : 83 Diff: 8.387800693512	
Top13 drop down : 23 Diff: 8.1821160316467	
Top14 drop down : 76 Diff: 8.1637177467346	
Top15 drop down : 61 Diff: 8.1610112190247	
Top16 drop down : 152 Diff: 8.1203656196594	
Top17 drop down : 239 Diff: 8.0883450508118	
Top18 drop down : 119 Diff: 8.0260128974915	
Top19 drop down : 87 Diff: 7.9801468849182	
Top20 drop down : 102 Diff: 7.918354511261	
Top21 drop down : 145 Diff: 7.911985874176	
Top22 drop down : 127 Diff: 7.8027062416077	
Top23 drop down : 150 Diff: 7.7

Top67 drop down : 35 Diff: 7.397509098053	
Top68 drop down : 49 Diff: 7.3973050117493	
Top69 drop down : 58 Diff: 7.3954358100891	
Top70 drop down : 116 Diff: 7.3939290046692	
Top71 drop down : 103 Diff: 7.390709400177	
Top72 drop down : 201 Diff: 7.3903107643127	
Top73 drop down : 173 Diff: 7.3897862434387	
Top74 drop down : 160 Diff: 7.3857941627502	
Top75 drop down : 52 Diff: 7.3852009773254	
Top76 drop down : 104 Diff: 7.3836102485657	
Top77 drop down : 200 Diff: 7.3832173347473	
Top78 drop down : 36 Diff: 7.3808636665344	
Top79 drop down : 125 Diff: 7.3720116615295	
Top80 drop down : 8 Diff: 7.3665871620178	
Top81 drop down : 161 Diff: 7.3524613380432	
Top82 drop down : 94 Diff: 7.3494057655334	
Top83 drop down : 50 Diff: 7.3477540016174	
Top84 drop down : 212 Diff: 7.3465180397034	
Top85 drop down : 120 Diff: 7.3394246101379	
Top86 drop down : 56 Diff: 7.3295559883118	
Top87 drop down : 171 Diff: 7.3287072181702	
Top88 drop down : 129 Diff: 7.3229603767395	
Top89 drop down : 107 

0436096	
Top158 drop down : 156 Diff: 7.2247586250305	
Top159 drop down : 151 Diff: 7.2245202064514	
Top160 drop down : 168 Diff: 7.2244973182678	
Top161 drop down : 20 Diff: 7.2225995063782	
Top162 drop down : 62 Diff: 7.2211060523987	
Top163 drop down : 245 Diff: 7.2197976112366	
Top164 drop down : 2 Diff: 7.2172265052795	
Top165 drop down : 19 Diff: 7.2167916297913	
Top166 drop down : 95 Diff: 7.2162957191467	
Top167 drop down : 122 Diff: 7.2157692909241	
Top168 drop down : 210 Diff: 7.2120938301086	
Top169 drop down : 128 Diff: 7.211136341095	
Top170 drop down : 24 Diff: 7.2083172798157	
Top171 drop down : 126 Diff: 7.2046475410461	
Top172 drop down : 118 Diff: 7.2045350074768	
Top173 drop down : 33 Diff: 7.1981415748596	
Top174 drop down : 237 Diff: 7.190887928009	
Top175 drop down : 225 Diff: 7.1908688545227	
Top176 drop down : 226 Diff: 7.1847043037415	
Top177 drop down : 183 Diff: 7.1839985847473	
Top178 drop down : 170 Diff: 7.1809086799622	
Top179 drop down : 131 Diff: 7.1742

463	
Top248 drop down : 66 Diff: 6.7133049964905	
Top249 drop down : 223 Diff: 6.7126851081848	
Top250 drop down : 137 Diff: 6.7119736671448	
Top251 drop down : 54 Diff: 6.7005486488342	
Top252 drop down : 77 Diff: 6.5640645027161	
Top253 drop down : 114 Diff: 6.5016865730286	
Top254 drop down : 190 Diff: 6.4226613044739	
Top255 drop down : 243 Diff: 6.3726468086243	
Top256 drop down : 165 Diff: 6.0985074043274	


In [71]:
for top_n=1, 256 do
    local incomplete_fm = net:get(1):get(13).output:clone()
    for i=1,net:get(1):get(13).output:size(1) do
        local drop = true
        for j=1,top_n do
            if i == topN_index[j].idx then
                drop = false
            end  
        end

        if drop == true then
            --print(i)
            incomplete_fm[i]:zero()
        end
    end

    sorted_prob, classes = classifier:forward(incomplete_fm):view(-1):float():sort(true)
    --output_org, average = NormalizedMax( sorted_prob )
    --local ratio_of_mean_error = (output_org - average)/average

    print('Top '..tostring(top_n)..' feature map classification: ', synset_words[classes[1]] .. ' ' .. sorted_prob[1] )
    
    --print(classes[1])
    if true_label == classes[1] then
        classification_curve[top_n]= sorted_prob[1] 
    else
        classification_curve[top_n] = 0
    end
end

Top 1 feature map classification: 	orange 6.2644243240356	


Top 2 feature map classification: 	goldfish, Carassius auratus 8.3550662994385	


Top 3 feature map classification: 	goldfish, Carassius auratus 13.201498031616	


Top 4 feature map classification: 	goldfish, Carassius auratus 15.004779815674	


Top 5 feature map classification: 	goldfish, Carassius auratus 17.022672653198	


...

Top 252 feature map classification: 	goldfish, Carassius auratus 25.307580947876	


Top 253 feature map classification: 	goldfish, Carassius auratus 24.783437728882	


Top 254 feature map classification: 	goldfish, Carassius auratus 23.896701812744	


Top 255 feature map classification: 	goldfish, Carassius auratus 23.036737442017	


Top 256 feature map classification: 	goldfish, Carassius auratus 21.853404998779	


In [72]:
--print(classification_curve)
point_index = torch.Tensor(classification_curve:size(1))  
for i=1, classification_curve:size(1) do
    point_index[i] = i
end
plot = Plot():line( point_index, classification_curve,'red','Classification Curve'):legend(true):title('Line Plot Demo'):draw()


--gnuplot.plot( classification_curve )

In [82]:
for top_n=1, 256 do
    local incomplete_fm = net:get(1):get(13).output:clone()
    for i=1,net:get(1):get(13).output:size(1) do
        local drop = false
        for j=1,top_n do
            if i == topN_index[j].idx then
                drop = true
            end  
        end

        if drop == true then
            --print(i)
            print('Drop neuron :'.. i)
            incomplete_fm[i]:zero()
        end
    end

    sorted_prob, classes = classifier:forward(incomplete_fm):view(-1):float():sort(true)
    --output_org, average = NormalizedMax( sorted_prob )
    --local ratio_of_mean_error = (output_org - average)/average

    print('Top '..tostring(top_n)..' feature map classification: ', synset_words[classes[1]] .. ' ' .. sorted_prob[1] )
    
    --print(classes[1])
    if true_label == classes[1] then
        classification_curve[top_n]= sorted_prob[1] 
    else
        classification_curve[top_n] = 0
    end
end

Drop neuron :153	


Top 1 feature map classification: 	goldfish, Carassius auratus 16.692687988281	
Drop neuron :12	


Drop neuron :153	


Top 2 feature map classification: 	goldfish, Carassius auratus 11.67681312561	


Drop neuron :12	
Drop neuron :153	
Drop neuron :179	


...

Top 255 feature map classification: 	punching bag, punch bag, punching ball, punchball 2.5886402130127	
Drop neuron :1	
Drop neuron :2	
Drop neuron :3	
Drop neuron :4	
Drop neuron :5	
Drop neuron :6	
Drop neuron :7	
Drop neuron :8	
Drop neuron :9	
Drop neuron :10	
Drop neuron :11	
Drop neuron :12	
Drop neuron :13	
Drop neuron :14	
Drop neuron :15	
Drop neuron :16	
Drop neuron :17	
Drop neuron :18	
Drop neuron :19	
Drop neuron :20	
Drop neuron :21	
Drop neuron :22	
Drop neuron :23	
Drop neuron :24	
Drop neuron :25	
Drop neuron :26	
Drop neuron :27	
Drop neuron :28	
Drop neuron :29	
Drop neuron :30	
Drop neuron :31	
Drop neuron :32	
Drop neuron :33	
Drop neuron :34	
Drop neuron :35	
Drop neuron :36	
Drop neuron :37	
Drop neuron :38	
Drop neuron :39	
Drop neuron :40	
Drop neuron :41	
Drop neuron :42	


Drop neuron :43	
Drop neuron :44	
Drop neuron :45	
Drop neuron :46	
Drop neuron :47	
Drop neuron :48	
Drop neuron :49	
Drop neuron :50	
Drop neuron :51	
Drop neuron :52	
Drop neuron :53	
Drop neuron :54	
Drop neuron :55	
Drop neuron :56	
Drop neuron :57	
Drop neuron :58	
Drop neuron :59	
Drop neuron :60	
Drop neuron :61	
Drop neuron :62	
Drop neuron :63	
Drop neuron :64	
Drop neuron :65	
Drop neuron :66	
Drop neuron :67	
Drop neuron :68	
Drop neuron :69	
Drop neuron :70	
Drop neuron :71	
Drop neuron :72	
Drop neuron :73	
Drop neuron :74	
Drop neuron :75	
Drop neuron :76	
Drop neuron :77	
Drop neuron :78	
Drop neuron :79	
Drop neuron :80	
Drop neuron :81	
Drop neuron :82	
Drop neuron :83	
Drop neuron :84	
Drop neuron :85	
Drop neuron :86	
Drop neuron :87	
Drop neuron :88	
Drop neuron :89	
Drop neuron :90	
Drop neuron :91	
Drop neuron :92	
Drop neuron :93	
Drop neuron :94	
Drop neuron :95	
Drop neuron :96	
Drop neuron :97	
Drop neuron :98	
Drop neuron :99	
Drop neuron :100	
Drop neuron 

Drop neuron :136	
Drop neuron :137	
Drop neuron :138	
Drop neuron :139	
Drop neuron :140	
Drop neuron :141	
Drop neuron :142	
Drop neuron :143	
Drop neuron :144	
Drop neuron :145	
Drop neuron :146	
Drop neuron :147	
Drop neuron :148	
Drop neuron :149	
Drop neuron :150	
Drop neuron :151	
Drop neuron :152	
Drop neuron :153	
Drop neuron :154	
Drop neuron :155	
Drop neuron :156	
Drop neuron :157	
Drop neuron :158	
Drop neuron :159	
Drop neuron :160	
Drop neuron :161	
Drop neuron :162	
Drop neuron :163	
Drop neuron :164	
Drop neuron :165	
Drop neuron :166	
Drop neuron :167	
Drop neuron :168	
Drop neuron :169	
Drop neuron :170	
Drop neuron :171	
Drop neuron :172	
Drop neuron :173	
Drop neuron :174	
Drop neuron :175	
Drop neuron :176	
Drop neuron :177	
Drop neuron :178	
Drop neuron :179	
Drop neuron :180	
Drop neuron :181	
Drop neuron :182	
Drop neuron :183	
Drop neuron :184	
Drop neuron :185	
Drop neuron :186	
Drop neuron :187	
Drop neuron :188	
Drop neuron :189	
Drop neuron :190	
Drop neuro

Drop neuron :238	
Drop neuron :239	
Drop neuron :240	
Drop neuron :241	
Drop neuron :242	
Drop neuron :243	
Drop neuron :244	
Drop neuron :245	
Drop neuron :246	
Drop neuron :247	
Drop neuron :248	
Drop neuron :249	
Drop neuron :250	
Drop neuron :251	
Drop neuron :252	
Drop neuron :253	
Drop neuron :254	
Drop neuron :255	
Drop neuron :256	


Top 256 feature map classification: 	shoe shop, shoe-shop, shoe store 0.024763148277998	


In [81]:
--print(classification_curve)
point_index = torch.Tensor(classification_curve:size(1))  
for i=1, classification_curve:size(1) do
    point_index[i] = i
end
plot = Plot():line( point_index, classification_curve,'red','Classification Curve'):legend(true):title('Line Plot Demo'):draw()


In [186]:
net

nn.Sequential {
  [input -> (1) -> (2) -> (3) -> output]
  (1): nn.Sequential {
    [input -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> (7) -> (8) -> (9) -> (10) -> (11) -> (12) -> (13) -> (14) -> output]
    (1): cudnn.SpatialConvolution(3 -> 96, 7x7, 2,2, 1,1)
    (2): cudnn.ReLU
    (3): inn.SpatialSameResponseNormalization
    (4): cudnn.SpatialMaxPooling
    (5): cudnn.SpatialConvolution(96 -> 256, 5x5, 2,2)
    (6): cudnn.ReLU
    (7): inn.SpatialSameResponseNormalization
    (8): cudnn.SpatialMaxPooling
    (9): cudnn.SpatialConvolution(256 -> 384, 3x3, 1,1, 1,1)
    (10): cudnn.ReLU
    (11): cudnn.SpatialConvolution(384 -> 384, 3x3, 1,1, 1,1)
    (12): cudnn.ReLU
    (13): cudnn.SpatialConvolution(384 -> 256, 3x3, 1,1, 1,1)
    (14): cudnn.ReLU
  }
  (2): inn.SpatialPyramidPooling
  (3): nn.Sequential {
    [input -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> (7) -> output]
    (1): nn.Linear(12800 -> 4096)
    (2): cudnn.ReLU
    (3): nn.Dropout(0.500000)
    (4): nn.Linear(40

              {
                  padW : 1
                  nInputPlane : 3
                  output : CudaTensor - size: 96x110x110
                  gradInput : CudaTensor - size: 3x224x224
                  iDesc : cdata<struct cudnnTensorStruct *[1]>: 0x40eb55d8
                  iSize : LongStorage - size: 4
                  convDesc : cdata<struct cudnnConvolutionStruct *[1]>: 0x41ef75a0
                  output_offset : 1161600
                  gradBias : CudaTensor - size: 96
                  algType : cdata<enum 761 [?]>: 0x4194c7d0
                  extraBuffer : CudaTensor - size: 147
                  extraBufferSizeInBytes : 588
                  oDescForBias : cdata<struct cudnnTensorStruct *[1]>: 0x416dc7c0
                  oDesc : cdata<struct cudnnTensorStruct *[1]>: 0x4089ff50
                  groups : 1
                  dH : 2
                  dW : 2
                  nOutputPlane : 96
 

                 bias : CudaTensor - size: 96
                  kH : 7
                  weight_offset : 14112
                  finput : CudaTensor - empty
                  biasDesc : cdata<struct cudnnTensorStruct *[1]>: 0x4030b1c0
                  input_offset : 150528
                  weight : CudaTensor - size: 96x3x7x7
                  train : false
                  gradWeight : CudaTensor - size: 96x3x7x7
                  fgradInput : CudaTensor - empty
                  kW : 7
                  padH : 1
                  weightDesc : cdata<struct cudnnFilterStruct *[1]>: 0x416b4a18
                }
              2 : 
                cudnn.ReLU
                {
                  inplace : true
                  mode : CUDNN_ACTIVATION_RELU
                  train : false
   

               output : CudaTensor - size: 96x110x110
                  gradInput : CudaTensor - empty
                  nElem : 1161600
                  iDesc : cdata<struct cudnnTensorStruct *[1]>: 0x407380a8
                }
              3 : 
                inn.SpatialSameResponseNormalization
                {
                  alpha : 5e-05
                  train : false
                  output : CudaTensor - size: 96x110x110
                  gradInput : CudaTensor - empty
                  size : 3
                  beta : 0.75
                  modules : nn.Sequential {
  [input -> (1) -> (2) -> output]
  (1): nn.ConcatTable {
    input
      |`-> (1): nn.Identity
      |`-> (2): nn.Sequential {
      |      [input -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> output]
      |      (1): nn.SpatialZeroPadding(l=1,r=1,t=1,b=1)
      |      (2): nn.Power
      |      (3): nn.SpatialAveragePooling(3,3,1,1)
      |      (4): nn.MulConstant
      |      (5): nn.AddConstant
      

            dH : 2
                  dW : 2
                  padH : 0
                  gradInput : CudaTensor - size: 96x110x110
                  oDesc : cdata<struct cudnnTensorStruct *[1]>: 0x408a1dd0
                  iSize : LongStorage - size: 4
                  iDesc : cdata<struct cudnnTensorStruct *[1]>: 0x40ea5468
                  poolDesc : cdata<struct cudnnPoolingStruct *[1]>: 0x40616c78
                  mode : CUDNN_POOLING_MAX
                  train : false
                  kW : 3
                  ceil_mode : true
                  padW : 0
                  output : CudaTensor - size: 96x55x55
                  kH : 3
                }
              5 : 
                cudnn.SpatialConvolution(96 -> 256, 5x5, 2,2)
                {
                  padW : 0
            

...

   4 : 
                {
                  1 : 6
                  2 : 6
                }
            }
          train : false
          output : CudaTensor - size: 12800
          gradInput : CudaTensor - empty
          module : 
            nn.Concat {
              input
                |`-> (1): nn.Sequential {
                |      [input -> (1) -> (2) -> output]
                |      (1): nn.SpatialAdaptiveMaxPooling
                |      (2): nn.View
                |    }
                |`-> (2): nn.Sequential {
                |      [input -> (1) -> (2) -> output]
                |      (1): nn.SpatialAdaptiveMaxPooling
                |      (2): nn.View
                |    }
                |`-> (3): nn.Sequential {
                |      [input -> (1) -> (2) -> output]
                |      (1): nn.SpatialAdaptiveMaxPooling
                |      (2): nn.View
                |    }
                |`-> (4): nn.Sequential {
                |      [input -> (1) -> 

        nn.Sequential {
          [input -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> (7) -> output]
          (1): nn.Linear(12800 -> 4096)
          (2): cudnn.ReLU
          (3): nn.Dropout(0.500000)
          (4): nn.Linear(4096 -> 4096)
          (5): cudnn.ReLU
          (6): nn.Dropout(0.500000)
          (7): nn.Linear(4096 -> 1000)
        }
        {
          gradInput : CudaTensor - empty
          modules : 
            {
              1 : 
                nn.Linear(12800 -> 4096)
                {
                  gradBias : CudaTensor - size: 4096
                  weight : CudaTensor - size: 4096x12800
                  train : false
                  bias : CudaTensor - size: 4096
                  gradInput : CudaTensor - empty
                  output : CudaTensor - size: 4096
                  gradWeight : CudaTensor - size: 4096x12800
                }
              2 : 
                cudnn.ReLU
                {
                  inplace : true
                

   nElem : 4096
                  iDesc : cdata<struct cudnnTensorStruct *[1]>: 0x40b338f8
                }
              3 : 
                nn.Dropout(0.500000)
                {
                  v2 : false
                  noise : CudaTensor - empty
                  train : false
                  p : 0.5
                  gradInput : CudaTensor - empty
                  output : CudaTensor - size: 4096
                }
              4 : 
                nn.Linear(4096 -> 4096)
                {
                  gradBias : CudaTensor - size: 4096
                  weight : CudaTensor - size: 4096x4096
                  train : false
                  bias : CudaTensor - size: 4096
                  gradInput : CudaTensor - empty
                  output : CudaTensor - size: 4096
                  gradWeight : CudaTensor - size: 4096x4096
          

      }
              5 : 
                cudnn.ReLU
                {
                  inplace : true
                  mode : CUDNN_ACTIVATION_RELU
                  train : false
                  output : CudaTensor - size: 4096
                  gradInput : CudaTensor - empty
                  nElem : 4096
                  iDesc : cdata<struct cudnnTensorStruct *[1]>: 0x419545b0
                }
              6 : 
                nn.Dropout(0.500000)
                {
                  v2 : false
                  noise : CudaTensor - empty
                  train : false
                  p : 0.5
                  gradInput : CudaTensor - empty
                  output : CudaTensor - size: 4096
                }
              7 : 
                nn.Linear(4096 -> 1000)
                {


                  gradBias : CudaTensor - size: 1000
                  weight : CudaTensor - size: 1000x4096
                  train : false
                  bias : CudaTensor - size: 1000
                  gradInput : CudaTensor - empty
                  output : CudaTensor - size: 1000
                  gradWeight : CudaTensor - size: 1000x4096
                }
            }
          train : false
          output : CudaTensor - size: 1000
        }
    }
  train : false
  output : CudaTensor - size: 1000
}


In [22]:
x1 = torch.randn(40)
y1 = torch.Tensor(40)
for i=1,40 do
    y1[i] = i 
end
print(x1)
plot = Plot():line(y1, x1, 'red','example'):legend(true):title('Line Plot Demo'):draw()

 0.9400
 0.1357
 1.1384
-0.9214
 0.6813
 0.2097
-1.0063
 0.4904
 0.7521
 0.1191
-0.7687
 0.1466
-1.1040
 1.4171
 1.2943
-0.4709
-0.3478
-0.2723
-1.7604
-0.8848
-1.3617
 0.9320
-0.5779
-0.7112
-0.8270
-0.4356
 0.1186
-0.4663
-0.8055
-0.9754
-1.2868
-0.5876
 0.2873
 0.4034
-0.1160
 1.3642
 0.0904
 1.4338
 2.6844
-1.0716
[torch.DoubleTensor of size 40]

